# Product Recommendation with Feathr on Azure (Advanced)

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

### Install Feathr python package and it's dependencies

Here, we install the package from the repository's main branch. To use the latest release, you may run `pip install feathr[notebook]` instead. If so, however, some of the new features in this notebook might not work.

In [1]:
# Uncomment and run this cell to install feathr from the latest codes in the repo. You may use `pip install feathr[notebook]` as well.
# !pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]"  

In [2]:
# We also install InterpretML package for model explainability
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 4.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.7 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 5.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.

## 2. Config Feathr Client

In [5]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame


import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df
from feathr.utils.platform import is_databricks


print(f"Feathr version: {feathr.__version__}")

Feathr version: 0.9.0


In [31]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "foobared" # default password for Redis

import tempfile
yaml_config = f"""
api_version: 1
project_config:
  project_name: 'product_recommendation'

spark_config:
  spark_cluster: 'local'
  spark_result_output_parts: '1'
  local:
    master: 'local[*]'
    feathr_runtime_location:

online_store:
  redis:
    # Redis configs to access Redis cluster
    host: '127.0.0.1'
    port: 6379
    ssl_enabled: False

feature_registry:
  # The API endpoint of the registry service
  api_endpoint: "http://127.0.0.1:8000/api/v1"
"""

tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)

### Generate the Feathr client configuration

The code below will write the onfiguration to a temporary location that will be used by a Feathr client. Please refer to [feathr_config.yaml](https://github.com/feathr-ai/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) for full list of configuration options and details.

### Initialize Feathr Client

In [32]:
client = FeathrClient(tmp.name)

2023-01-14 05:36:18.729 | INFO     | feathr.utils._env_config_reader:get:65 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:36:18.732 | INFO     | feathr.utils._env_config_reader:get:65 - Config offline_store__s3__s3_enabled is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:36:18.732 | INFO     | feathr.utils._env_config_reader:get:65 - Config offline_store__adls__adls_enabled is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:36:18.733 | INFO     | feathr.utils._env_config_reader:get:65 - Config offline_store__wasb__wasb_enabled is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:36:18.734 | INFO     | feathr.utils._env_config_reader:get:65 - Config offline_store__jdbc__jdbc_enabled is not found in the environment variable

## 3. Prepare Datasets

1. Download datasets
2. Upload to cloud storage if necessary so that the target cluster can consume them as the data sources

In [33]:
# Use dbfs if the notebook is running on Databricks
if is_databricks():
    WORKING_DIR = f"/dbfs/{PROJECT_NAME}"
else:
    WORKING_DIR = PROJECT_NAME

In [34]:
# Download datasets
user_observation_file_path = f"{WORKING_DIR}/user_observation.csv"
user_profile_file_path = f"{WORKING_DIR}/user_profile.csv"
user_purchase_history_file_path = f"{WORKING_DIR}/user_purchase_history.csv"
product_detail_file_path = f"{WORKING_DIR}/product_detail.csv"
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    dst_filepath=user_observation_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    dst_filepath=user_profile_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    dst_filepath=user_purchase_history_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
    dst_filepath=product_detail_file_path,
)

False

In [35]:
# Upload files to cloud if needed
if client.spark_runtime == "local":
    # In local mode, we can use the same data path as the source.
    # If the notebook is running on databricks, DATA_FILE_PATH should be already a dbfs path.
    user_observation_source_path = user_observation_file_path
    user_profile_source_path = user_profile_file_path
    user_purchase_history_source_path = user_purchase_history_file_path
    product_detail_source_path = product_detail_file_path
elif client.spark_runtime == "databricks" and is_databricks():
    # If the notebook is running on databricks, we can use the same data path as the source.
    user_observation_source_path = user_observation_file_path.replace("/dbfs", "dbfs:")
    user_profile_source_path = user_profile_file_path.replace("/dbfs", "dbfs:")
    user_purchase_history_source_path = user_purchase_history_file_path.replace("/dbfs", "dbfs:")
    product_detail_source_path = product_detail_file_path.replace("/dbfs", "dbfs:")
else:
    # Otherwise, upload the local file to the cloud storage (either dbfs or adls).
    user_observation_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_observation_file_path)
    user_profile_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_profile_file_path)
    user_purchase_history_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_purchase_history_file_path)
    product_detail_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(product_detail_file_path)

## 3. Define Sharable Features using Feathr API

### Understand raw datasets
We have three datasets to work with:
* Observation dataset (a.k.a. labeled dataset)
* User profile
* User purchase history
* Product details

In [36]:
# Observation dataset
# Observation dataset usually comes with a event_timestamp to denote when the observation happened.
# The label here is product_rating. Our model objective is to predict a user's rating for this product.
pd.read_csv(user_observation_file_path).head()

,user_id,product_id,event_timestamp,product_rating
0,1,1,2021-04-01,4
1,1,2,2021-04-01,4
2,1,3,2021-04-01,4
3,1,4,2021-04-01,4
4,1,5,2021-04-01,4


In [37]:
# User profile dataset
# Used to generate user features
pd.read_csv(user_profile_file_path).head()

,user_id,gender,age,gift_card_balance,number_of_credit_cards,state,tax_rate
0,1,1,22,100,0,CA,7.5
1,2,2,17,300,1,CA,7.5
2,3,1,40,0,2,WA,7.5
3,4,1,25,100,3,WA,7.5
4,5,1,33,0,2,PA,0.0


In [38]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to genearte features.
pd.read_csv(user_purchase_history_file_path).head()

,user_id,purchase_date,purchase_amount
0,1,2021-01-01,0.33
1,1,2021-03-03,574.35
2,1,2021-01-03,796.07
3,2,2021-01-04,342.15
4,2,2021-03-05,280.46


In [39]:
# Product detail dataset.
# Used to generate product features.
pd.read_csv(product_detail_file_path).head()

,product_id,category,price,quantity,recent_sold,made_in_state,discount
0,1,1,22,100,0,CA,7.5
1,2,2,17,300,1,CA,7.5
2,3,1,40,0,2,WA,7.5
3,4,1,25,100,3,WA,7.5
4,5,1,33,0,2,PA,0.0


### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive. 

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields. 

In [40]:
def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col

    return df.withColumn("tax_rate_decimal", col("tax_rate") / 100)


batch_source = HdfsSource(
    name="userProfileData",
    path=user_profile_source_path,
    preprocessing=feathr_udf_preprocessing,
)

In [41]:
# Let's define some features for users so our recommendation can be customized for users.
user_id = TypedKey(
    key_column="user_id",
    key_column_type=ValueType.INT32,
    description="user id",
    full_name="product_recommendation.user_id",
)

feature_user_age = Feature(
    name="feature_user_age",
    key=user_id,
    feature_type=INT32,
    transform="age",
)
feature_user_tax_rate = Feature(
    name="feature_user_tax_rate",
    key=user_id,
    feature_type=FLOAT,
    transform="tax_rate_decimal",
)
feature_user_gift_card_balance = Feature(
    name="feature_user_gift_card_balance",
    key=user_id,
    feature_type=FLOAT,
    transform="gift_card_balance",
)
feature_user_has_valid_credit_card = Feature(
    name="feature_user_has_valid_credit_card",
    key=user_id,
    feature_type=BOOLEAN,
    transform="number_of_credit_cards > 0",
)

features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card,
]

user_feature_anchor = FeatureAnchor(
    name="anchored_features", source=batch_source, features=features
)

In [42]:
# Let's define some features for the products so our recommendation can be customized for proudcts.
product_batch_source = HdfsSource(
    name="productProfileData",
    path=product_detail_source_path,
)

product_id = TypedKey(
    key_column="product_id",
    key_column_type=ValueType.INT32,
    description="product id",
    full_name="product_recommendation.product_id",
)

feature_product_quantity = Feature(
    name="feature_product_quantity",
    key=product_id,
    feature_type=FLOAT,
    transform="quantity",
)
feature_product_price = Feature(
    name="feature_product_price", key=product_id, feature_type=FLOAT, transform="price"
)

product_features = [feature_product_quantity, feature_product_price]

product_feature_anchor = FeatureAnchor(
    name="product_anchored_features",
    source=product_batch_source,
    features=product_features,
)

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

In [43]:
purchase_history_data = HdfsSource(
    name="purchase_history_data",
    path=user_purchase_history_source_path,
    event_timestamp_column="purchase_date",
    timestamp_format="yyyy-MM-dd",
)

agg_features = [
    Feature(
        name="feature_user_avg_purchase_for_90days",
        key=user_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(purchase_amount)", agg_func="AVG", window="90d"
        ),
    )
]

user_agg_feature_anchor = FeatureAnchor(
    name="aggregationFeatures", source=purchase_history_data, features=agg_features
)

### Derived Features Section
Derived features are the features that are computed from other features. They could be computed from anchored features or other derived features.

In [44]:
derived_features = [
    DerivedFeature(
        name="feature_user_purchasing_power",
        key=user_id,
        feature_type=FLOAT,
        input_features=[feature_user_gift_card_balance, feature_user_has_valid_credit_card],
        transform="feature_user_gift_card_balance + if(boolean(feature_user_has_valid_credit_card), 100, 0)",
    )
]

### Build features

Lastly, we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features which is not anchored to a source.

In [45]:
client.build_features(
    anchor_list=[user_agg_feature_anchor, user_feature_anchor, product_feature_anchor],
    derived_feature_list=derived_features,
)

## 4. Create Training Data using Point-in-Time Correct Feature join

To create a training dataset using Feathr, we need to provide a **feature join settings** to specify what features and how these features should be joined to the observation data. 

Also note that since a `FeatureQuery` accepts features of the same join key, we define two query objects, one for `user_id` key and the other one for `product_id` and pass them together to compute offline features. 

To learn more on this topic, please refer to [Point-in-time Correctness document](https://feathr-ai.github.io/feathr/concepts/point-in-time-join.html).

In [46]:
user_feature_query = FeatureQuery(
    feature_list=[feat.name for feat in features + agg_features + derived_features],
    key=user_id,
)

product_feature_query = FeatureQuery(
    feature_list=[feat.name for feat in product_features],
    key=product_id,
)

settings = ObservationSettings(
    observation_path=user_observation_source_path,
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd",
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=[user_feature_query, product_feature_query],
    output_path=user_profile_source_path.rpartition("/")[0] + f"/product_recommendation_features.avro",
)
client.wait_job_to_finish(timeout_sec=5000)

2023-01-14 05:36:24.712 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2023-01-14 05:36:24.714 | INFO     | feathr.spark_provider._localspark_submission:_init_args:225 - Spark job: product_recommendation_feathr_feature_join_job is running on local spark with master: local[*].
2023-01-14 05:36:24.718 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:121 - Detail job stdout and stderr are in debug/product_recommendation_feathr_feature_join_job20230114053624/log.
2023-01-14 05:36:24.719 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:131 - Local Spark job submit with pid: 6058.
2023-01-14 05:36:24.720 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:141 - 1 local spark job(s) in this Launcher, only the latest will be monitored.
2023-01-14 05:36:24.721 | INFO     | feathr.spark_

debug/product_recommendation_feathr_feature_join_job20230114053624
['/tmp/tmp9ffeol29/feathr_pyspark_driver.py']
x>>>>>>>>>>>>>>>

2023-01-14 05:36:46.752 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:168 - Pyspark job Completed


>

2023-01-14 05:36:47.756 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:191 - Spark job with pid 6058 finished in: 23 seconds                     with returncode 143


>

Let's use the helper function `get_result_df` to download the result and view it:

In [47]:
res_df = get_result_df(client)
res_df.head()

2023-01-14 05:36:54.930 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:141 - 1 local spark job(s) in this Launcher, only the latest will be monitored.
2023-01-14 05:36:54.932 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:142 - Please check auto generated spark command in debug/product_recommendation_feathr_feature_join_job20230114053624/command.sh and detail logs in debug/product_recommendation_feathr_feature_join_job20230114053624/log.
2023-01-14 05:36:54.933 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:191 - Spark job with pid 6058 finished in: 0 seconds                     with returncode 143


,user_id,product_id,event_timestamp,product_rating,feature_user_avg_purchase_for_90days,feature_product_price,feature_product_quantity,feature_user_gift_card_balance,feature_user_has_valid_credit_card,feature_user_tax_rate,feature_user_age,feature_user_purchasing_power
0,1,1,2021-04-01,4,456.916656,22.0,100.0,100.0,False,0.075,22,100.0
1,1,2,2021-04-01,4,456.916656,17.0,300.0,100.0,False,0.075,22,100.0
2,1,3,2021-04-01,4,456.916656,40.0,0.0,100.0,False,0.075,22,100.0
3,1,4,2021-04-01,4,456.916656,25.0,100.0,100.0,False,0.075,22,100.0
4,1,5,2021-04-01,4,456.916656,33.0,0.0,100.0,False,0.075,22,100.0


### Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr. Here, we use **EBM (Explainable Boosting Machine)** regressor from [InterpretML](https://github.com/interpretml/interpret) package to visualize the modeling results.

In [48]:
from interpret import show
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


# Fill None values with 0
final_df = (
    res_df
    .drop(["event_timestamp"], axis=1, errors="ignore")
    .fillna(0)
)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop(["product_rating"], axis=1),
    final_df["product_rating"].astype("float64"),
    test_size=0.2,
    random_state=42,
)

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

# Note, currently InterpretML's visualization dashboard doesn't work w/ VSCODE notebook viewer
# https://github.com/interpretml/interpret/issues/317
ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/140366732200784/ -->

In [49]:
# Predict and evaluate
y_pred = ebm.predict(X_test)
rmse = sqrt(mean_squared_error(y_test.values.flatten(), y_pred))

print(f"Root mean squared error: {rmse}")

Root mean squared error: 0.22016160907715643


## 5. Feature Materialization

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:

In [50]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings(
    name="user_feature_setting",
    sinks=[redisSink],
    feature_names=["feature_user_age", "feature_user_gift_card_balance"],
)

client.materialize_features(settings=settings, allow_materialize_non_agg_feature=True)
client.wait_job_to_finish(timeout_sec=5000)

2023-01-14 05:37:06.567 | INFO     | feathr.utils._env_config_reader:get:65 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:37:06.569 | INFO     | feathr.utils._env_config_reader:get:65 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:37:06.571 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2023-01-14 05:37:06.572 | INFO     | feathr.spark_provider._localspark_submission:_init_args:225 - Spark job: product_recommendation_feathr_feature_materialization_job is running on local spark with master: local[*].
2023-01-14 05:37:06.576 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:121 - Detail job stdout and stderr are in debug/product_recom

debug/product_recommendation_feathr_feature_materialization_job20230114053706
['/tmp/tmp9ffeol29/feathr_pyspark_driver.py']
x>x>>>>>

2023-01-14 05:37:25.604 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:168 - Pyspark job Completed


>

2023-01-14 05:37:26.609 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:191 - Spark job with pid 7607 finished in: 20 seconds                     with returncode 143


>

We can then get the features from the online store (Redis):

In [51]:
client.get_online_features(
    "user_features", "2", ["feature_user_age", "feature_user_gift_card_balance"]
)

[17, 300.0]

In [52]:
client.multi_get_online_features(
    "user_features", ["1", "2"], ["feature_user_age", "feature_user_gift_card_balance"]
)

{'1': [22, 100.0], '2': [17, 300.0]}

We can also materialize product features into a separate table.

In [53]:
# Materialize product features
backfill_time = BackfillTime(
    start=datetime(2020, 5, 20),
    end=datetime(2020, 5, 20),
    step=timedelta(days=1),
)

redisSink = RedisSink(table_name="product_features")
settings = MaterializationSettings(
    "product_feature_setting",
    backfill_time=backfill_time,
    sinks=[redisSink],
    feature_names=["feature_product_price"],
)

client.materialize_features(settings, allow_materialize_non_agg_feature=True)
client.wait_job_to_finish(timeout_sec=5000)

2023-01-14 05:37:33.477 | INFO     | feathr.utils._env_config_reader:get:65 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:37:33.478 | INFO     | feathr.utils._env_config_reader:get:65 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store.
2023-01-14 05:37:33.479 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2023-01-14 05:37:33.480 | INFO     | feathr.spark_provider._localspark_submission:_init_args:225 - Spark job: product_recommendation_feathr_feature_materialization_job is running on local spark with master: local[*].
2023-01-14 05:37:33.482 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:94 - Main JAR file is not set, using default package 'com.lin

debug/product_recommendation_feathr_feature_materialization_job20230114053733
x>>>>>>>>>>xxx

2023-01-14 05:38:08.528 | WARNING  | feathr.spark_provider._localspark_submission:wait_for_completion:153 - Spark job has hang for 15 seconds. latest msg is 23/01/14 05:37:48 INFO DAGScheduler: Job 7 finished: save at RedisOutputUtils.scala:37, took 0.159421 s
.                             Please check debug/product_recommendation_feathr_feature_materialization_job20230114053733/log_2.txt
2023-01-14 05:38:08.528 | WARNING  | feathr.spark_provider._localspark_submission:_clean_up:198 - Terminate the spark job due to as clean_up is set to True.
2023-01-14 05:38:08.624 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:191 - Spark job with pid 8851 finished in: 35 seconds                     with returncode 143


In [54]:
client.get_online_features("product_features", "2", ["feature_product_price"])

[17.0]

## 6. Feature Registration
Lastly, we can also register the features and share them across teams:

In [56]:
try:
    client.register_features()
except Exception as e:
    print(e)
print(client.list_registered_features(project_name=PROJECT_NAME))

[{'name': 'feature_user_age', 'id': '09422a93-a452-4e93-8059-f58b5be5e5f8', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_age'}, {'name': 'feature_user_tax_rate', 'id': '16c8b3e2-e757-4c1e-ac5b-9680cce80567', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_tax_rate'}, {'name': 'feature_product_price', 'id': '7c264bca-cdb1-409d-9583-5532a64d6451', 'qualifiedName': 'product_recommendation__product_anchored_features__feature_product_price'}, {'name': 'feature_user_has_valid_credit_card', 'id': '8a97434e-63f5-40fd-9062-09c75e5f1804', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_has_valid_credit_card'}, {'name': 'feature_user_avg_purchase_for_90days', 'id': '9290137e-9828-4f57-837f-f51c46491939', 'qualifiedName': 'product_recommendation__aggregationFeatures__feature_user_avg_purchase_for_90days'}, {'name': 'feature_user_gift_card_balance', 'id': '9827ac9d-b6ae-49fa-b134-15c106a2d19b', 'qualifiedName': 'produc

## Cleanup

In [57]:
# Cleaning up the output files. CAUTION: this maybe dangerous if you "reused" the project name.
import shutil
shutil.rmtree(WORKING_DIR, ignore_errors=False)

Scrap Variables for Unit-Test

In [58]:
if SCRAP_RESULTS:
    # Record results for test pipelines
    import scrapbook as sb
    sb.glue(
        "user_features",
        client.get_online_features(
            "user_features", "2", ["feature_user_age", "feature_user_gift_card_balance"]
        ),
    )
    sb.glue(
        "product_features",
        client.get_online_features(
            "product_features", "2", ["feature_product_price"]
        ),
    )
    
    sb.glue("rmse", rmse)